In [2]:
import csv
import os
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from token import EXACT_TOKEN_TYPES
from sklearn.metrics import accuracy_score # Accuracy metrics 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
import pickle

Capturar marcadores y exportar a .CSV

In [2]:
num_coords = 42
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [3]:
with open('Avianca2.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
variable = None

In [ ]:
for etapa in range(1,7):
    cap = cv2.VideoCapture("Videos-home/Paso"+str(etapa)+".mp4")#Videos-home/Paso1.mp4
    with mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands = 2, ) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
                break
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            #image.flags.writeable = True
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            #Draw the hand annotations on the image.
            #image.flags.writeable = False
            #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            height, width, _ = image.shape

            if results.multi_hand_landmarks:

                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=3),
                    mp_drawing.DrawingSpec(color=(255,0,255), thickness=2, circle_radius=3))

                # verificar primero la cantidad de landmarks
                if len(results.multi_hand_landmarks) == 1:
                    variable =results.multi_handedness[0].classification[0].label
                elif len(results.multi_hand_landmarks) == 2:
                    variable = 'Both'
                else:
                    variable = None

                #try:
                if variable =="Left":
                    mano_dere_row = np.zeros(63)
                    mano_izq_row = np.array([[landmark.x, landmark.y,landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten()
                elif variable == "Right":
                    mano_izq_row = np.zeros(63)
                    mano_dere_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten()
                elif variable == "Both":
                    mano_dere_row = np.array([[landmark.x, landmark.y,landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten()
                    mano_izq_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.multi_hand_landmarks[1].landmark]).flatten()
                else:
                    print('Error, se reconocen mas de dos manos')
                print("va en el paso", etapa)
                rows = np.concatenate((np.array([etapa]),mano_dere_row,mano_izq_row))
                print(rows)
                rows = np.where(rows ==0, np.nan,rows)
                centroid_x = np.nanmean(rows[1::3])
                centroid_y = np.nanmean(rows[2::3])
                centroid_z = np.nanmean(rows[3::3])
                rows = np.where(np.isnan(rows), 0, rows)
                rows[1::3] -= centroid_x
                rows[2::3] -= centroid_y
                rows[3::3] -= centroid_z
                    
                with open('Avianca2.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(rows) 

            cv2.imshow('Paso '+str(etapa), image)
            if cv2.waitKey(5) & 0xFF == 27:
                break
        cap.release()
        cv2.destroyAllWindows()

In [2]:
#por si toca entrenar el modelo en tiempo real
cap = cv2.VideoCapture(0)#Videos-home/Paso1.mp4
with mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands = 2, ) as hands:
    while cap.isOpened():
        etapa = 1
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
            break
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        #image.flags.writeable = True
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        #Draw the hand annotations on the image.
        #image.flags.writeable = False
        #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        height, width, _ = image.shape

        if results.multi_hand_landmarks:

            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=3),
                mp_drawing.DrawingSpec(color=(255,0,255), thickness=2, circle_radius=3))

            # verificar primero la cantidad de landmarks
            if len(results.multi_hand_landmarks) == 1:
                variable =results.multi_handedness[0].classification[0].label
            elif len(results.multi_hand_landmarks) == 2:
                variable = 'Both'
            else:
                variable = None

            #try:
            if variable =="Left":
                mano_dere_row = np.zeros(63)
                mano_izq_row = np.array([[landmark.x, landmark.y,landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten()
            elif variable == "Right":
                mano_izq_row = np.zeros(63)
                mano_dere_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten()
            elif variable == "Both":
                mano_dere_row = np.array([[landmark.x, landmark.y,landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten()
                mano_izq_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.multi_hand_landmarks[1].landmark]).flatten()
            else:
                print('Error, se reconocen mas de dos manos')
            print("va en el paso", etapa)
            rows = np.concatenate((np.array([etapa]),mano_dere_row,mano_izq_row))
            print(rows)
            rows = np.where(rows ==0, np.nan,rows)
            centroid_x = np.nanmean(rows[1::3])
            centroid_y = np.nanmean(rows[2::3])
            centroid_z = np.nanmean(rows[3::3])
            rows = np.where(np.isnan(rows), 0, rows)
            rows[1::3] -= centroid_x
            rows[2::3] -= centroid_y
            rows[3::3] -= centroid_z
                
            with open('Avianca2.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(rows) 

        cv2.imshow('Paso '+str(etapa), image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
hand_language_class = model.predict(X)[0]

            #print(hand_language_class)
        hand_language_prob = model.predict_proba(X)[0]

            #print(hand_language_class, hand_language_prob)

            #poner caja estatica
        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

        cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(hand_language_class)
                       , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Probability
        cv2.putText(image, 'PROB'
                      , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(hand_language_prob[np.argmax(hand_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

ENTRENAR EL MODELO CON SCIKIT LEARN

In [4]:
df = pd.read_csv('Avianca2.csv',index_col=False)
df.head()

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z39,x40,y40,z40,x41,y41,z41,x42,y42,z42
0,1.0,-0.454182,-0.534350,0.007788,-0.454182,-0.534350,0.007788,-0.454182,-0.534350,0.007788,...,0.010978,0.044009,0.059554,0.008346,0.053766,0.042641,0.005303,0.060688,0.028643,0.003077
1,1.0,-0.460229,-0.532614,0.008409,-0.460229,-0.532614,0.008409,-0.460229,-0.532614,0.008409,...,0.014450,0.046485,0.063264,0.012876,0.058515,0.042297,0.010040,0.067686,0.021963,0.007812
2,1.0,0.095499,0.177505,0.018624,0.049926,0.200893,-0.001201,0.014612,0.167119,-0.011307,...,0.025880,0.023672,0.060534,0.026913,0.034890,0.033012,0.025191,0.043195,0.006125,0.023128
3,1.0,-0.459766,-0.524205,0.005092,-0.459766,-0.524205,0.005092,-0.459766,-0.524205,0.005092,...,0.007144,0.046917,0.061571,0.005857,0.057741,0.035386,0.002858,0.065292,0.008912,-0.000082
4,1.0,-0.460934,-0.520516,0.004127,-0.460934,-0.520516,0.004127,-0.460934,-0.520516,0.004127,...,0.007623,0.046893,0.055749,0.006801,0.057585,0.025734,0.003160,0.064913,-0.003697,-0.000479


In [5]:
df.shape

(14774, 127)

In [6]:
y = df['class'].values  #targe vaue
x = df.drop(['class'],axis=1).values # caracterisiticas 

In [7]:
param_grid = {'n_estimators':[10,20,80,100,500], 
            'criterion':['gini'], 
            'min_samples_split':[2,5,20,80,100,200],
            'max_features':['auto']
             }

In [8]:
grid = GridSearchCV(
        estimator  = RandomForestClassifier(random_state = 123),
        param_grid = param_grid,
        n_jobs     = multiprocessing.cpu_count() - 1,
        verbose    = 0,
        return_train_score = True
       )

In [9]:
grid.fit(X = x, y = y)

/Users/juannquinones/opt/anaconda3/envs/HigieneManos/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/juannquinones/opt/anaconda3/envs/HigieneManos/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/juannquinones/opt/anaconda3/envs/HigieneManos/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the 

GridSearchCV(estimator=RandomForestClassifier(random_state=123), n_jobs=7,
             param_grid={'criterion': ['gini'], 'max_features': ['auto'],
                         'min_samples_split': [2, 5, 20, 80, 100, 200],
                         'n_estimators': [10, 20, 80, 100, 500]},
             return_train_score=True)

In [10]:
results=(pd.DataFrame(grid.cv_results_))
best_results=results.sort_values(by='mean_test_score', ascending=False).head(20)[['params','mean_test_score']]

In [11]:
best_results['params'].values

array([{'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 500},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 500},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 80},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 100},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 80},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 100},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 20, 'n_estimators': 500},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 20, 'n_estimators': 100},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 20, 'n_estimators': 80},
       {'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 20},
       {'cr

In [12]:
clf = RandomForestClassifier(random_state = 123)
clf.set_params(**{'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 20, 'n_estimators': 500})
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123,stratify=y)
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

/Users/juannquinones/opt/anaconda3/envs/HigieneManos/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.9898488608166027

In [13]:
clf.predict(x_test)

array([2., 4., 6., ..., 2., 1., 1.])

In [14]:
clf = RandomForestClassifier(random_state = 123)
clf.set_params(**{'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 20, 'n_estimators': 500})
clf.fit(x,y)
with open('avianca.pkl', 'wb') as f:
    pickle.dump(clf, f)

/Users/juannquinones/opt/anaconda3/envs/HigieneManos/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


duración video de entrenamiento 
Paso 1: 4 sec  127
Paso 2: 5 sec
Paso 3: 3 sec
Paso 4: 3 sec
Paso 5: 5 sec
Paso 6: 4 sec

In [24]:
can_clases = df.iloc[:, 0] # Primera columna
len(can_clases)

719

In [44]:

que_clase = df.iloc[ :, 0] 
que_clase.value_counts()

2    148
5    144
6    127
1    114
4     95
3     91
Name: class, dtype: int64

Poner lo de Over Sampling